# Multilingual semantic search

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/main/blob/notebooks/search/04-multilingual.ipynb)


 이 예에서는 다국어 임베딩 모델을 사용합니다.
[multilingual-e5-base](https://huggingface.co/intfloat/multilingual-e5-base)는 혼합된 간단한 데이터세트에서 검색을 수행합니다.
언어 문서. 이 모델을 사용하면 두 가지 방법으로 검색할 수 있습니다.
  * 여러 언어에 걸쳐(예: 독일어로 된 쿼리를 사용하여 영어로 된 문서 찾기)
  * 영어가 아닌 언어 내에서(예: 독일어로 된 문서를 찾기 위해 독일어로 된 쿼리 사용)

이 예에서는 밀집 검색만 사용하고 있지만 밀집 검색과 전통적 어휘 검색을 결합하는 것도 가능합니다.
하이브리드 검색으로 어휘 다국어 검색에 대한 자세한 내용은 블로그 게시물을 참조하세요.
[Elasticsearch의 언어 식별을 이용한 다국어 검색](https://www.elastic.co/blog/multilingual-search-using-언어-identification-in-elasticsearch).

사용된 데이터 세트에는 [MIRACL](https://project-miracl.github.io/) 데이터 세트의 Wikipedia 구절 스니펫이 포함되어 있습니다.

# 🧰 Requirements

이 예에서는 다음이 필요합니다.

- 파이썬 3.6 이상
- 최소 **4GB 기계 학습 노드**
- [Elastic Python 클라이언트](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/installation.html)


# Install packages and initialize the Elasticsearch Python client

시작하려면 Python 클라이언트를 사용하여 Elastic 배포에 연결해야 합니다.

먼저 이 예제에 필요한 패키지를 `pip` 설치해야 합니다.

In [ ]:
!pip install elasticsearch
!pip install sentence_transformers
!pip install torch

다음으로 `elasticsearch` 모듈과 `getpass` 모듈을 가져와야 합니다.
`getpass`는 Python 표준 라이브러리의 일부이며 자격 증명을 안전하게 요청하는 데 사용됩니다.

In [ ]:
import getpass
import textwrap
import torch

from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer("intfloat/multilingual-e5-base", device=device)

이제 Python Elasticsearch 클라이언트를 인스턴스화할 수 있습니다.
먼저 사용자에게 비밀번호와 Cloud ID를 묻는 메시지를 표시합니다.

🔐 참고: `getpass`를 사용하면 자격 증명을 터미널에 표시하거나 메모리에 저장하지 않고 사용자에게 자격 증명을 안전하게 묻는 메시지를 표시할 수 있습니다.

그런 다음 `Elasticsearch` 클래스의 인스턴스를 인스턴스화하는 `클라이언트` 개체를 만듭니다.

In [ ]:
from getpass import getpass

ES_URL = getpass('Elasticsearch URL: ')
ES_ID = "elastic" 
ES_PW = getpass('elastic user PW: ')
CERT_PATH = getpass('Elasticsearch cer 파일 경로: ')

# Create the client instance
client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_ID, ES_PW),
    ca_certs=CERT_PATH
)

Confirm that the client has connected with this test

In [ ]:
print(client.info())

Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Create Elasticsearch index with required mappings

We need to add a field to support dense vector storage and search.
Note the `passage_embedding` field below, which is used to store the dense vector representation of the `passage` field.

In [ ]:
# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "language": {"type": "keyword"},
            "id": {"type": "keyword"},
            "title": {"type": "text"},
            "passage": {"type": "text"},
            "passage_embedding": {
                "type": "dense_vector",
                "dims": 768,
                "index": "true",
                "similarity": "cosine"
            }
        }
    }
}

# Create the index (deleting any existing index)
client.indices.delete(index="articles", ignore_unavailable=True)
client.indices.create(index="articles", body=mapping)


## Dataset

Let's index some data.
Note that we are embedding the `passage` field using the sentence transformer model.
Once indexed, you'll see that your documents contain a `passage_embedding` field (`"type": "dense_vector"`) which contains a vector of floating point values.
This is the embedding of the `passage` field in vector space.
We'll use this field to perform semantic search using kNN.

In [ ]:
articles = [
    {
        "language": "en",
        "id": "1643584#0",
        "title": "Bloor Street",
        "passage": """Bloor Street is a major east–west residential and commercial thoroughfare in Toronto, Ontario, Canada. Bloor Street runs from the Prince Edward Viaduct, which spans the Don River Valley, westward into Mississauga where it ends at Central Parkway. East of the viaduct, Danforth Avenue continues along the same right-of-way. The street, approximately long, contains a significant cross-sample of Toronto's ethnic communities. It is also home to Toronto's famous shopping street, the Mink Mile.""",
    },
    {
        "language": "en",
        "id": "2190499#0",
        "title": "Elphinstone College",
        "passage": """Elphinstone College is an institution of higher education affiliated to the University of Mumbai. Established in 1856, it is one of the oldest colleges of the University of Mumbai. It is reputed for producing luminaries like Bal Gangadhar Tilak, Bhim Rao Ambedkar, Virchand Gandhi, Badruddin Tyabji, Pherozshah Mehta, Kashinath Trimbak Telang, Jamsetji Tata and for illustrious professors that includes Dadabhai Naoroji. It is further observed for having played a key role in spread of Western education in the Bombay Presidency.""",
    },
    {
        "language": "en",
        "id": "8881#0",
        "title": "Doctor (title)",
        "passage": """Doctor is an academic title that originates from the Latin word of the same spelling and meaning. The word is originally an agentive noun of the Latin verb "" 'to teach'. It has been used as an academic title in Europe since the 13th century, when the first Doctorates were awarded at the University of Bologna and the University of Paris. Having become established in European universities, this usage spread around the world. Contracted "Dr" or "Dr.", it is used as a designation for a person who has obtained a Doctorate (e.g. PhD). In many parts of the world it is also used by medical practitioners, regardless of whether or not they hold a doctoral-level degree.""",
    },
    {
        "language": "de",
        "id": "9002#0",
        "title": "Gesundheits- und Krankenpflege",
        "passage": """Die Gesundheits- und Krankenpflege als Berufsfeld umfasst die Versorgung und Betreuung von Menschen aller Altersgruppen, insbesondere kranke, behinderte und sterbende Erwachsene. Die Gesundheits- und Kinderkrankenpflege hat ihren Schwerpunkt in der Versorgung von Kindern und Jugendlichen. In beiden Fachrichtungen gehört die Verhütung von Krankheiten und Gesunderhaltung zum Aufgabengebiet der professionellen Pflege.""",
    },
    {
        "language": "de",
        "id": "7769762#0",
        "title": "Tourismusregion (Österreich)",
        "passage": """Unter Tourismusregion versteht man in Österreich die in den Landestourismusgesetzen verankerten Tourismusverbände mehrerer Gemeinden, im weiteren Sinne aller Gebietskörperschaften.""",
    },
    {
        "language": "de",
        "id": "2270104#0",
        "title": "London Wall",
        "passage": """London Wall ist die strategische Stadtmauer, die die Römer um Londinium gebaut haben, um die Stadt zu schützen, die über den wichtigen Hafen an der Themse verfügte. Bis ins späte Mittelalter hinein bildete diese Stadtmauer die Grenzen von London. Heute ist "London Wall" auch der Name einer Straße, die an einem noch bestehenden Abschnitt der Stadtmauer verläuft.""",
    },
    {
        "language": "de",
        "id": "2270104#1",
        "title": "London Wall",
        "passage": """Die Mauer wurde Ende des zweiten oder Anfang des dritten Jahrhunderts erbaut, wahrscheinlich zwischen 190 und 225, vermutlich zwischen 200 und 220. Sie entstand somit etwa achtzig Jahre nach dem im Jahr 120 erfolgten Bau der Festung, deren nördliche und westliche Mauern verstärkt und in der Höhe verdoppelt wurden, um einen Teil der neuen Stadtmauer zu bilden. Die Anlage wurde zumindest bis zum Ende des vierten Jahrhunderts weiter ausgebaut. Sie zählt zu den letzten großen Bauprojekten der Römer vor deren Rückzug aus Britannien im Jahr 410.""",
    },
]

## Index documents

Our dataset is a Python list that contains dictionaries of passages from Wikipedia articles in two languages.
We'll use the `helpers.bulk` method to index our documents in batches.

The following code iterates over the articles and creates a list of actions to be performed.
Each action is a dictionary containing an "index" operation on our Elasticsearch index.
The passage is encoded using our selected model, and the encoded vector is added to the article document.
Note that the E5 models require that a prefix instruction is used "passage: " to tell the model that it is to embed a passage.
On the query side, the query string will be prefixed with "query: ".
The article document is then added to the list of actions.

Finally, we call the `bulk` method, specifying the index name and the list of actions.

In [ ]:
actions = []
for article in articles:
    actions.append({"index": {"_index": "articles"}})
    passage = article["passage"]
    passageEmbedding = model.encode(f"passage: {passage}").tolist()
    article["passage_embedding"] = passageEmbedding
    actions.append(article)

client.bulk(index="articles", operations=actions)

# Multilingual Semantic Search

In the following, we will search using two kinds of queries:
 * Query in English to find documents in any language
 * Query in German to find documents in German only (using a filter),
   to show the model's capabilities in non-English languages

Note again that the query is prefixed with "query: ", which the model requires to encode the query properly.

A quick translation for those unfamiliar with German:
 * "health" -> "Gesundheit"
 * "wall" -> "Mauer"

In [ ]:
def pretty_response(response):
    for hit in response["hits"]["hits"]:
        score = hit["_score"]
        language = hit["_source"]["language"]
        id = hit["_source"]["id"]
        title = hit["_source"]["title"]
        passage = hit["_source"]["passage"]
        print()
        print(f"ID: {id}")
        print(f"Language: {language}")
        print(f"Title: {title}")
        print(f"Passage: {textwrap.fill(passage, 120)}")
        print(f"Score: {score}")

In [ ]:
def query(q, language=None):
    knn = {
        "field": "passage_embedding",
        "query_vector" : model.encode(f"query: {q}").tolist(),
        "k": 2,
        "num_candidates": 5
    }

    if language:
        knn["filter"] = {
            "term": {
                "language": language,
            }
        }

    return client.search(index="articles", knn=knn)

In [13]:
pretty_response(query("health"))


ID: 9002#0
Language: de
Title: Gesundheits- und Krankenpflege
Passage: Die Gesundheits- und Krankenpflege als Berufsfeld umfasst die Versorgung und Betreuung von Menschen aller Altersgruppen,
insbesondere kranke, behinderte und sterbende Erwachsene. Die Gesundheits- und Kinderkrankenpflege hat ihren Schwerpunkt
in der Versorgung von Kindern und Jugendlichen. In beiden Fachrichtungen gehört die Verhütung von Krankheiten und
Gesunderhaltung zum Aufgabengebiet der professionellen Pflege.
Score: 0.8986236

ID: 8881#0
Language: en
Title: Doctor (title)
Passage: Doctor is an academic title that originates from the Latin word of the same spelling and meaning. The word is originally
an agentive noun of the Latin verb "" 'to teach'. It has been used as an academic title in Europe since the 13th
century, when the first Doctorates were awarded at the University of Bologna and the University of Paris. Having become
established in European universities, this usage spread around the world. Contract

Note that in the results above, we see that the document about healthcare,
even though it's in German, matches better to the query "health",
versus the English document which doesn't talk about health specifically but about doctors more generally.
This is the power of a multilingual embedding which embeds meaning across languages.

In [14]:
pretty_response(query("wall", language="de"))


ID: 2270104#0
Language: de
Title: London Wall
Passage: London Wall ist die strategische Stadtmauer, die die Römer um Londinium gebaut haben, um die Stadt zu schützen, die über
den wichtigen Hafen an der Themse verfügte. Bis ins späte Mittelalter hinein bildete diese Stadtmauer die Grenzen von
London. Heute ist "London Wall" auch der Name einer Straße, die an einem noch bestehenden Abschnitt der Stadtmauer
verläuft.
Score: 0.8941859

ID: 2270104#1
Language: de
Title: London Wall
Passage: Die Mauer wurde Ende des zweiten oder Anfang des dritten Jahrhunderts erbaut, wahrscheinlich zwischen 190 und 225,
vermutlich zwischen 200 und 220. Sie entstand somit etwa achtzig Jahre nach dem im Jahr 120 erfolgten Bau der Festung,
deren nördliche und westliche Mauern verstärkt und in der Höhe verdoppelt wurden, um einen Teil der neuen Stadtmauer zu
bilden. Die Anlage wurde zumindest bis zum Ende des vierten Jahrhunderts weiter ausgebaut. Sie zählt zu den letzten
großen Bauprojekten der Römer vor der

In [15]:
pretty_response(query("Mauer", language="de"))


ID: 2270104#1
Language: de
Title: London Wall
Passage: Die Mauer wurde Ende des zweiten oder Anfang des dritten Jahrhunderts erbaut, wahrscheinlich zwischen 190 und 225,
vermutlich zwischen 200 und 220. Sie entstand somit etwa achtzig Jahre nach dem im Jahr 120 erfolgten Bau der Festung,
deren nördliche und westliche Mauern verstärkt und in der Höhe verdoppelt wurden, um einen Teil der neuen Stadtmauer zu
bilden. Die Anlage wurde zumindest bis zum Ende des vierten Jahrhunderts weiter ausgebaut. Sie zählt zu den letzten
großen Bauprojekten der Römer vor deren Rückzug aus Britannien im Jahr 410.
Score: 0.88160384

ID: 2270104#0
Language: de
Title: London Wall
Passage: London Wall ist die strategische Stadtmauer, die die Römer um Londinium gebaut haben, um die Stadt zu schützen, die über
den wichtigen Hafen an der Themse verfügte. Bis ins späte Mittelalter hinein bildete diese Stadtmauer die Grenzen von
London. Heute ist "London Wall" auch der Name einer Straße, die an einem noch bestehe